In [ ]:
!pip install transformers gradio langchain sentence-transformers langchain_chroma

!pip install langchain-community

# !pip install chromadb

In [ ]:
!pip install --force-reinstall pandas
!pip install --force-reinstall gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 23.5 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.2
    Uninstalling pytz-2025.2:
      Successfully uninstalled pytz-2025.2
  Attempting uninstall: tzdata
    Found existing installation: tzdata 2025.2
    Uninstalling tzdata-2025.2:
      Successfully uninstalled tzdata-2025.2
  Attempting uninstall: six
    Found existing installation: six 1.17.0
    Uninstalling six-1.17.0:
      Successfully uninstalled six-1

  Using cached gradio-5.23.3-py3-none-any.whl.metadata (16 kB)
  Using cached aiofiles-23.2.1-py3-none-any.whl.metadata (9.7 kB)
  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached ffmpy-0.5.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached gradio_client-1.8.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached groovy-0.1.2-py3-none-any.whl.metadata (6.1 kB)
ERROR: Operation cancelled by user
^C


In [ ]:
!unzip /content/Knowledge-base.zip -d /content/

Archive:  /content/Knowledge-base.zip
   creating: /content/Knowledge-base/
   creating: /content/Knowledge-base/disease-info/
  inflating: /content/Knowledge-base/disease-info/disease_treatment_info_expanded.md  
  inflating: /content/Knowledge-base/disease-info/first_aid_emergency.md  
  inflating: /content/Knowledge-base/disease-info/health_tips_preventive_care.md  
  inflating: /content/Knowledge-base/disease-info/medication_info_expanded.md  


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `chatbot` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate 

In [ ]:
import os
import glob
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings

# Configuration
DB_NAME = "vector_db"
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
model_name = "meta-llama/Llama-3.2-1B"

# Load the tokenizer and model for text generation
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.float16, device_map="auto"
)

# Set up the pipeline for text generation
text_gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.9
)

# Load documents from the knowledge base
folders = glob.glob("/content/Knowledge-base/*")
text_loader_kwargs = {'encoding': 'utf-8'}
documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()

    if folder_docs:
        documents.extend(folder_docs)
    else:
        print(f"Warning: No documents found in {folder}")

print(f"Loaded {len(documents)} documents")

# Split text into chunks for processing
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

# Create the embeddings
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)

# Initialize Chroma for vector storage
if os.path.exists(DB_NAME):
    Chroma(persist_directory=DB_NAME, embedding_function=embeddings).delete_collection()

# Create and persist vector store
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=DB_NAME)

# Create a retriever from the vector store
retriever = vectorstore.as_retriever()

# Memory setup for conversation history
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# Define the chat function for conversation and context retrieval
def chat(question, history):
    # Retrieve relevant context from the vector store
    context = retriever.get_relevant_documents(question)

    if not context:
        context_text = "No relevant context found. Answer to the best of your ability."
    else:
        context_text = "\n".join([doc.page_content for doc in context])

    # Prepare input text for the model
    input_text = f"### Context:\n{context_text}\n\n### Question:\n{question}\n\n### Answer:"

    print(f"Debug - Input to model: {input_text}")  # Debugging output

    # Generate the model response
    response = text_gen_pipeline(input_text)[0]["generated_text"]

    # Clean up the response to remove input_text
    return response.replace(input_text, "").strip()

# Set up Gradio interface for chat
view = gr.ChatInterface(chat, type="messages").launch(debug=True, inbrowser=True)
